# Tracking MCMC Workflow
This notebook attempts to track the workflow of MCMC. This notebook has MANY purposes.

First, we inject a signal. For the injection, MCMC uses the following workflow.

MCMC calls ===> LALInferenceInjectInspiralSignal which calls ===>
[XLALSimInspiralChooseTDWaveform](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__c.html#gaf8e3028cce411328ddc1a3efa6f43cb2)

The fref of the waveform is 100 unless --inj-fref = ? is specified.

This then passes the inj file parameters through the following function 

[XLALSimInspiralTransformPrecessingInitialConditions](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gadc3d6ebbe9e62c8d7ea93b489db00bdb)

All that happens here is that S1,S2 get two ROTATEY(inc,S1,S2) (inj.inclination stays the same in orbital L frame)

Explicitally, The SpinTaylorT4 section looks as follows

 * 558             /* Sanity check unused fields of waveFlags */
 * 559             spin1[0]=S1x; spin1[1]=S1y; spin1[2]=S1z;
 * 560             spin2[0]=S2x; spin2[1]=S2y; spin2[2]=S2z;
 * 561             iTmp=i;
 * 562          XLALSimInspiralInitialConditionsPrecessingApproxs(&i,&S1x,&S1y,&S1z,&S2x,&S2y,&S2z,iTmp,spin1[0],spin1[1],spin1[2],spin2[0],spin2[1],spin2[2],m1,m2,f_ref,XLALSimInspiralGetFrameAxis(waveFlags));
 * 563             LNhatx = sin(i);
 * 564             LNhaty = 0.;
 * 565             LNhatz = cos(i);
 * 566             E1x = cos(i);
 * 567             E1y = 0.;
 * 568             E1z = - sin(i);
 * 569             /* Maximum PN amplitude order for precessing waveforms is
 * 570              * MAX_PRECESSING_AMP_PN_ORDER */
 * 571             amplitudeO = amplitudeO <= MAX_PRECESSING_AMP_PN_ORDER ?
 * 572                     amplitudeO : MAX_PRECESSING_AMP_PN_ORDER;
 * 573             /* Call the waveform driver routine */
 * 574             ret = XLALSimInspiralSpinTaylorT4(hplus, hcross, phiRef, v0, deltaT,
 * 575                     m1, m2, f_min, f_ref, r, S1x, S1y, S1z, S2x, S2y, S2z,
 * 576                     LNhatx, LNhaty, LNhatz, E1x, E1y, E1z, lambda1, lambda2,
 * 577                     quadparam1, quadparam2,
 * 578                     XLALSimInspiralGetSpinOrder(waveFlags),
 * 579                     XLALSimInspiralGetTidalOrder(waveFlags),
 * 580                     phaseO, amplitudeO);

MCMC uses the template function below to generate the template waveform.

[LALInferenceTemplateXLALSimInspiralChooseWaveform](http://software.ligo.org/docs/lalsuite/lalinference/group___l_a_l_inference_template__h.html#ga8725ff30be0c84ea6be1e1cfb3a317ea)

In order to determine what iota S1 and S2 the precessing approximate has so it can generate the waveform it uses [XLALSimInspiralTransformPrecessingNewInitialConditions](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_spin_taylor__c.html#gacde647cce8c72a41d1dd0ac9e9b385da) which takes in inputs such as theta_jn phi_jl tilt1 tilt2 etc 
	
Inclination is determined as follows

 *                Starting frame: LNhat is along the z-axis and the unit
 * 3379           spin vectors are defined from the angles relative to LNhat.
 * 3380           Note that we put s1hat in the x-z plane, and phi12
 * 3381           sets the azimuthal angle of s2hat measured from the x-axis.
 * 3382          
 * 3383         LNhx = 0.;
 * 3384         LNhy = 0.;
 * 3385         LNhz = 1.;
 * 3386         /* Spins are given wrt to L, but still we cannot fill the spin as we do not know
 * 3387          * what will be the relative orientation of L and N.
 * 3388          */
 * 3389         s1hatx = sin(theta1);
 * 3390         s1haty = 0.;
 * 3391         s1hatz = cos(theta1);
 * 3392         s2hatx = sin(theta2) * cos(phi12);
 * 3393         s2haty = sin(theta2) * sin(phi12);
 * 3394         s2hatz = cos(theta2);
 * 3395 
 * 3396         /* Define several internal variables needed for magnitudes */
 * 3397         m1 = m1_SI/LAL_MSUN_SI;
 * 3398         m2 = m2_SI/LAL_MSUN_SI;
 * 3399         eta=m1*m2/(m1+m2)/(m1+m2);
 * 3400         // v parameter at reference point
 * 3401         v0 = cbrt( (m1+m2) * LAL_MTSUN_SI *LAL_PI * fRef );
 * 3402 
 * 3403         /* Define S1, S2, J with proper magnitudes */
 * 3404         Lmag = XLALSimInspiralLN(m1+m2,eta,v0)*(1+v0*v0*XLALSimInspiralL_2PN(eta));
 * 3405         s1x = m1 * m1 * chi1 * s1hatx;
 * 3406         s1y = m1 * m1 * chi1 * s1haty;
 * 3407         s1z = m1 * m1 * chi1 * s1hatz;
 * 3408         s2x = m2 * m2 * chi2 * s2hatx;
 * 3409         s2y = m2 * m2 * chi2 * s2haty;
 * 3410         s2z = m2 * m2 * chi2 * s2hatz;
 * 3411         Jx = s1x + s2x;
 * 3412         Jy = s1y + s2y;
 * 3413         Jz = Lmag * LNhz + s1z + s2z;
 * 3414 
 * 3415         /* Normalize J to Jhat, find it's angles in starting frame */
 * 3416         Jnorm = sqrt( Jx*Jx + Jy*Jy + Jz*Jz);
 * 3417         Jhatx = Jx / Jnorm;
 * 3418         Jhaty = Jy / Jnorm;
 * 3419         Jhatz = Jz / Jnorm;
 * 3420         theta0 = acos(Jhatz);
 * 3421         phi0 = atan2(Jhaty, Jhatx);
 * 3422 
 * 3423         /* Rotation 1: Rotate about z-axis by -phi0 to put Jhat in x-z plane */
 * 3424         ROTATEZ(-phi0, LNhx, LNhy, LNhz);
 * 3425         ROTATEZ(-phi0, s1hatx, s1haty, s1hatz);
 * 3426         ROTATEZ(-phi0, s2hatx, s2haty, s2hatz);
 * 3427 
 * 3428         /* Rotation 2: Rotate about new y-axis by -theta0
 * 3429          * to put Jhat along z-axis
 * 3430          */
 * 3431         ROTATEY(-theta0, LNhx, LNhy, LNhz);
 * 3432         ROTATEY(-theta0, s1hatx, s1haty, s1hatz);
 * 3433         ROTATEY(-theta0, s2hatx, s2haty, s2hatz);
 * 3434 
 * 3435         /* Rotation 3: Rotate about new z-axis by phiJL to put L at desired
 * 3436          * azimuth about J. Note that is currently in x-z plane towards -x
 * 3437          * (i.e. azimuth=pi). Hence we rotate about z by phiJL - LAL_PI
 * 3438          */
 * 3439         ROTATEZ(phiJL - LAL_PI, LNhx, LNhy, LNhz);
 * 3440         ROTATEZ(phiJL - LAL_PI, s1hatx, s1haty, s1hatz);
 * 3441         ROTATEZ(phiJL - LAL_PI, s2hatx, s2haty, s2hatz);
 * 3442 
 * 3443         /* The cosinus of the angle between L and N is the scalar
 * 3444          * product of the two vectors. We do need to perform additional
 * 3445          * rotation to compute it
 * 3446          */
 * 3447         *incl=acos(-sin(thetaJN)*LNhx+cos(thetaJN)*LNhz); //output
 * 3448 
 * 3449         /* Rotation 4-5: Now J is along z and N in x-z plane, inclined from J
 * 3450          * by thetaJN. Now we bring L into the z axis to get
 * 3451          * spin components.
 * 3452          */
 * 3453         REAL8 Nx=-sin(thetaJN);
 * 3454         REAL8 Ny=0.;
 * 3455         REAL8 Nz=cos(thetaJN);
 * 3456         REAL8 thetaLJ = acos(LNhz);
 * 3457         REAL8 phiL    = atan2(LNhy, LNhx);
 * 3458 
 * 3459         ROTATEZ(-phiL, Nx, Ny, Nz);
 * 3460         ROTATEZ(-phiL, s1hatx, s1haty, s1hatz);
 * 3461         ROTATEZ(-phiL, s2hatx, s2haty, s2hatz);
 * 3462         ROTATEY(-thetaLJ, Nx, Ny, Nz);
 * 3463         ROTATEY(-thetaLJ, s1hatx, s1haty, s1hatz);
 * 3464         ROTATEY(-thetaLJ, s2hatx, s2haty, s2hatz);
 * 3465 
 * 3466         /* Rotation 6: Now L is along z and we have to bring N
 * 3467          * in the x-z plane.
 * 3468          */
 * 3469         REAL8 phiN = atan2(Ny, Nx);
 * 3470         ROTATEZ(-phiN, s1hatx, s1haty, s1hatz);
 * 3471         ROTATEZ(-phiN, s2hatx, s2haty, s2hatz);
 * 3472 
 * 3473         /* Set pointers to rotated spin vectors */
 * 3474         *S1x = s1hatx*chi1;
 * 3475         *S1y = s1haty*chi1;
 * 3476         *S1z = s1hatz*chi1;
 * 3477         *S2x = s2hatx*chi2;
 * 3478         *S2y = s2haty*chi2;
 * 3479         *S2z = s2hatz*chi2;
 
It then calls [XLALSimInspiralChooseTDWaveformFromCache](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral_waveform_cache__h.html#ga21112bbad5c769738c19ddb56701e6ae) to generate the waveform.


This function calls [XLALSimInspiralChooseTDWaveform](http://software.ligo.org/docs/lalsuite/lalsimulation/group___l_a_l_sim_inspiral__c.html#gaf8e3028cce411328ddc1a3efa6f43cb2)